In [ ]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import utils

In [ ]:
AUDIO_DIR = 'dataset/fma_small'

In [ ]:
# features = utils.load('dataset/fma_metadata/features.csv')
# features.head()

In [ ]:
tracks = pd.read_csv('project_data/tracks.csv', index_col=0)
track_ids = tracks.index

In [ ]:
track_id = 2
path = utils.get_audio_path(AUDIO_DIR, 2)

# Load an audio file
y, sr = librosa.load(path, sr=None)

# Extract MFCCs
mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)

# Display the MFCCs
plt.figure(figsize=(10, 4))
librosa.display.specshow(mfccs, x_axis='time', sr=sr)
plt.colorbar()
plt.title('MFCC')
plt.tight_layout()
plt.show()

In [ ]:
# Compute stats across time axis
mfcc_mean = np.mean(mfccs, axis=1)
mfcc_min = np.min(mfccs, axis=1)
mfcc_max = np.max(mfccs, axis=1)

# Print or use the statistics
for i in range(len(mfcc_mean)):
    print(
        f"MFCC {i+1:02}: Mean={mfcc_mean[i]:.2f}, Min={mfcc_min[i]:.2f}, Max={mfcc_max[i]:.2f}")

In [ ]:
# Create MultiIndex for columns
multi_col_index = pd.MultiIndex.from_product(
    [range(13), ['mean', 'min', 'max']], names=['mfcc_coeff', 'stat']
)

# Create DataFrame with MultiIndex columns and track_id as row index
mfcc_df = pd.DataFrame(columns=multi_col_index, index=track_ids)

cnt = 0

for i in track_ids:
    cnt += 1
    if cnt % 100 == 0:
        print(f"Processing track {cnt}...")
    path = utils.get_audio_path(AUDIO_DIR, i)
    y, sr = librosa.load(path, sr=None)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)

    mfcc_mean = np.mean(mfccs, axis=1)
    mfcc_min = np.min(mfccs, axis=1)
    mfcc_max = np.max(mfccs, axis=1)

    data = np.stack([mfcc_mean, mfcc_min, mfcc_max], axis=1).flatten()

    mfcc_df.loc[i] = data

In [ ]:
mfcc_df.head()

In [ ]:
mfcc_df.to_csv('project_data/mfcc_data.csv', index=True)

In [ ]:
mfcc_df = utils.load_mfcc_data()